In [22]:
import sys
sys.path.append('..')
from synthweave.utils.datasets import get_datamodule, get_dataset
from src.pipe import ImagePreprocessor, AudioPreprocessor

DATASET

In [23]:
ds_kwargs = {
    "data_dir": "../encoded_data/SWAN_DF_320",
    "preprocessed": True,
    "sample_mode": "single",
    "av_codes": ["00", "11"]
}

train = get_dataset("SWAN_DF", split="train", **ds_kwargs)
dev = get_dataset("SWAN_DF", split="dev", **ds_kwargs)
test = get_dataset("SWAN_DF", split="test", **ds_kwargs)

In [24]:
train[0]['video'].shape, train[0]['audio'].shape

(torch.Size([512]), torch.Size([192]))

In [25]:
train[0]['metadata']

{'label': np.int64(1),
 'av': np.int64(1),
 'id_source': '00010',
 'id_target': '00047'}

In [26]:
# vid_proc = ImagePreprocessor(window_len=4, step=1, head_pose_dir='../../../models/head_pose')
# aud_proc = AudioPreprocessor(window_len=4, step=1)

# ds_kwargs = {
#     'video_processor': vid_proc, 'audio_processor': aud_proc, 'mode': 'full'
# }

# ds = get_dataset("DeepSpeak_v1_1", split="train", **ds_kwargs)

### DATALOADER

In [27]:
dm = get_datamodule(
    "SWAN_DF",
    batch_size=4,
    dataset_kwargs=ds_kwargs,
    sample_mode="single",  # single, sequence
    clip_mode="id",  # 'id', 'idx'
    clip_to=2,  # 'min', int
    clip_selector="random",  # 'first', 'random'
)

# Example: max first 2 single window samples per id in batch

dm.setup("fit")

In [28]:
train_loader = dm.train_dataloader()
sample = next(iter(train_loader))

### EXAMPLE BATCH

In [29]:
sample["video"].shape

torch.Size([4, 512])

In [30]:
sample["audio"].shape

torch.Size([4, 192])

In [31]:
sample["metadata"]["label"]

tensor([0, 1, 1, 1])

In [32]:
dm.train_dataset.encoders["label"].inverse_transform(sample["metadata"]["label"])

array(['0', '1', '1', '1'], dtype='<U1')

In [33]:
sample["metadata"]["av"]

tensor([0, 1, 1, 1])

In [34]:
dm.train_dataset.encoders["av"].inverse_transform(sample["metadata"]["av"])

array(['00', '11', '11', '11'], dtype='<U2')